In [1]:
def introspect(obj):
  for func in [type, id, dir, vars, callable]:
        print("%s(%s):\t\t%s" % (func.__name__, introspect.__code__.co_varnames[0], func(obj)))


In [ ]:
import requests
import json
import hashlib
import os
from datetime import datetime as datingdays
import time
from pytz import timezone
from git import Repo, Git
import sys
import threading
project_root_path = '../../..'
python_lib_path = project_root_path + '/python/lib'
sys.path.append(python_lib_path)
from commit_log_parser import NumstatRequirementSet
from os.path import exists
from kitchen_sink_class import Query


q = Query()

class DoSomething:
    def doIt(self):

        if 0 == 1:    
            with open('./repo_classes.json', "r") as r:
                array = json.load(r)
        else:
            array = {}
            with open('./new_repo.log', 'r') as r:
                for l in r.readlines():
                    key = l[:-1] #Strip off carriage return
                    s = key.split('/')
                    array[key] = {'owner':s[0], 'repo_name':s[1], 'repo_full_name':key, 'count':0}
        q.preload_alias_maps()

        for n in array.values():
            owner = n['owner']
            repo_name = n['repo_name']
            repo = RepoName(owner, repo_name)
            print('Processing', owner, repo_name)
            repo_base_dir = './repos'
            result_base_dir = './results'
            repo_path = repo_base_dir+'/'+repo.key()
            result_path = result_base_dir+'/'+repo.key()
            json_stats_file_name = result_path+'/commit_stat_log.json'
        #    stat_req_set = StatRequirementSet()
            numstat_req_set = NumstatRequirementSet()
            last_date = datingdays.fromisoformat('1972-12-26T03:23:01.123456-07:00')

            if (os.path.isdir(repo_base_dir) == False):
                print('######### Cannot find '+repo_base_dir+'  Creating it!')
                os.makedirs(repo_base_dir)
            if os.path.isdir(result_base_dir) == False:
                print('Cannot find',result_base_dir,'Creating it!')
                os.makedirs(result_base_dir)
            if os.path.isdir(result_base_dir+'/'+owner) == False:
                os.makedirs(result_base_dir+'/'+owner)
            if os.path.isdir(result_base_dir+'/'+owner+'/'+repo_name) == False:
                os.makedirs(result_base_dir+'/'+owner+'/'+repo_name)
            if (os.path.isdir(repo_base_dir+"/"+owner) == False):
                os.makedirs(repo_base_dir+"/"+owner)
            url = 'https://github.com/'+owner+'/'+repo_name+'.git'
            cache_date = None
            if (os.path.isdir(repo_path) == False):
                Repo.clone_from(url, repo_path)
            else:
                rp = Repo(repo_path)
                remote = rp.remote()
                remote.pull()
                if exists(json_stats_file_name):
                    try:
                        cache_date = os.path.getmtime(json_stats_file_name)
                        with open(json_stats_file_name) as j:
                            numstat_req_set.resultArray = json.load(j)
                        for item in numstat_req_set.resultArray:
                            q.commit_cache_map[item['commit']] = item
                    except Exception as e:
                        cache_date = None
                        print('Error encountered trying to parse', json_stats_file_name, e)

            need_stats = True
            rep = Git(repo_path)
            if cache_date is not None:
                system_tz = timezone(time.tzname[0])        
                then = datingdays.now(system_tz)
                file_date = datingdays.fromtimestamp(cache_date, tz=system_tz)

                # Add call to rep.log('-1') to get the date from the latest change
                #  If that date is less than the date on the cached stats file
                #  then skip this one by loading the previous stats file.
                info = rep.log('-1')
                for n in info.splitlines():
                    prefix = 'Date: '
                    if n.startswith(prefix):
                        new_date = n[len(prefix):].strip()
                        dt = datingdays.strptime(new_date, '%a %b %d %H:%M:%S %Y %z')
                        then = then - (then - dt)
                        print(file_date.isoformat(), 'Last stats run')
                        print(then.isoformat(), 'Last Git Modification')
                        if then < file_date:
                            need_stats = False

                #print(info)
                #Parse the line that starts with Date
                #Date:   Mon May 16 19:14:08 2022 +0200

            if need_stats:        
                print('Generating Stats for '+repo_path)
                stat = rep.log('--numstat')
                numstat_req_set.processDocument(stat)

            q.repos[repo.owner+'/'+repo.repo_name] = numstat_req_set.resultArray.copy()

            if repo.owner not in q.json_repo_map.keys():
                q.json_repo_map[repo.owner] = {}
            q.json_repo_map[repo.owner][repo.repo_name] = q.repos[repo.key()]
            with open(json_stats_file_name, 'w') as out:
                out.write(json.dumps(numstat_req_set.resultArray, indent=2))
            for rae in numstat_req_set.resultArray:
                commit_id = rae['commit']
                alias = rae['Author']
                q.add_alias(alias, commit_id)
                q.add_commit_id(commit_id, repo)

        print('Done loading!')

        q.resolve_aliases()

        with open('./aliasMap.json', 'w') as out:
            out.write(json.dumps(q.resolved_alias_map, indent=2))

        with open('./new_repos.json', 'w') as out:
            out.write(json.dumps(q.repos, indent=2))
        with open('./hackers.json', 'w') as out:
            out.write(json.dumps(q.hackers, indent=2))

        for alias in q.aliases.keys():
            v = q.aliases[alias]
            print('Alias ', alias, ' has ', len(v), ' commits')

        print('How many hackers?', len(q.hackers))    
        repo_counter = {}
        call_count = 0
        if 1 == 0:
            with open('./new_repo.log', 'w') as new_repo_log:
                for hacker in q.hackers:
                    done = False
                    q.reset_last_date()
                    last_count = -1
                    while not done:
                        body = q.load_hacker_url(hacker)
                        call_count += 1
                        if call_count % 25 == 0:
                            print(call_count, 'rest API calls made')

                        if (body == None):
                            print('Unable to load JSON')
                            done = True
                        else:
                            total_count = body['total_count']
                            if (total_count == last_count):
                                print('Identical result set found.  Moving on.', total_count, last_count)
                                done = True
                            else:
                                print(total_count, 'remaining commits for user', hacker)
                            last_count = total_count
                            if total_count > 20000:
                                print('Yikes!', total_count, ' seems like a few too many')
                                done = True
                            incomplete_results = body['incomplete_results']
                #            print(total_count)
                #            print(incomplete_results)
                            array = body['items'];
                            if (array == None or len(array) < 1):
                                done = True
                            else:
                                for n in array:
                                    repo = n['repository']
                                    repo_full_name = repo['full_name']
                                    counter = None
                                    if repo_full_name not in repo_counter:
                                        counter = RepoCounter(repo)
                                        repo_counter[repo_full_name] = counter
                                        print('New repo found!', repo_full_name)
                                        new_repo_log.write(repo_full_name+'\n')
                                    else:
                                        counter = repo_counter[repo_full_name]
                                    counter.add_one()

                                    commit = n['commit']
                                    comAuth = commit['author']
                                    q.set_last_date(comAuth['date'])
                                if (total_count < 100 and incomplete_results == False):
                                    done = True

try:
    DoSomething().doIt()
except KeyboardInterrupt as ki:
    print('Keyboard Interrupt!  Leaving...')
except Exception as e:
    print('Unexpected error encountered:', e)
finally:
    q.stop_monitor()

try:
    with open('./hackers.json', 'w') as out:
        out.write(json.dumps(q.hackers, indent=2))
except:
    print('Unable to write out ./hackers.json')

try:    
    with open('./repo_classes.json', 'w') as out:
        out.write(json.dumps(repo_counter,default=lambda o: o.__dict__, 
                sort_keys=True,indent=2))
except:
    print('Unable to write out ./repo_classes.json')


    
